## Campaña de Marketing Bancario


> ### Comprensión empresarial
>
>Los depósitos a largo plazo permiten a los bancos retener dinero durante un período de tiempo específico, lo que permite al >banco utilizar ese dinero para mejorar sus inversiones. Las campañas de marketing de este producto se basan en llamadas >telefónicas. Si un usuario no se encuentra disponible en un momento dado, entonces se le volverá a llamar de nuevo en otro >momento.
>
>### Descripción del problema
>
>El banco portugués está teniendo una disminución en sus ingresos, por lo que quieren poder identificar a los clientes >existentes que tienen una mayor probabilidad de contratar un depósito a largo plazo. Esto permitirá que el banco centre sus >esfuerzos de marketing en esos clientes y evitará perder dinero y tiempo en clientes que probablemente no se suscribirán.
>
>Para abordar este problema crearemos un algoritmo de clasificación que ayude a predecir si un cliente contratará o no un >depósito a largo plazo.

## Recopilación de Datos

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# split para modelado
from sklearn.model_selection import train_test_split
# Scaled | Escalado
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Encoding | Codificación
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
# To save models
import json
import pickle
# Feature Selection
from sklearn.feature_selection import f_classif, SelectKBest

In [2]:
df_inicio = pd.read_csv('/workspaces/sarayruiz---Machine-Learning/data/raw/bank-marketing-campaign-data.csv', sep=';')
df_inicio


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


## Analisis descriptivo

In [3]:
df_inicio.shape

(41188, 21)

In [4]:
df_inicio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [5]:
df_inicio.describe().T

,count,mean,std,min,25%,50%,75%,max
age,41188.0,40.024060,10.421250,17.000,32.000,38.000,47.000,98.000
duration,41188.0,258.285010,259.279249,0.000,102.000,180.000,319.000,4918.000
campaign,41188.0,2.567593,2.770014,1.000,1.000,2.000,3.000,56.000
pdays,41188.0,962.475454,186.910907,0.000,999.000,999.000,999.000,999.000
previous,41188.0,0.172963,0.494901,0.000,0.000,0.000,0.000,7.000
emp.var.rate,41188.0,0.081886,1.570960,-3.400,-1.800,1.100,1.400,1.400
cons.price.idx,41188.0,93.575664,0.578840,92.201,93.075,93.749,93.994,94.767
cons.conf.idx,41188.0,-40.502600,4.628198,-50.800,-42.700,-41.800,-36.400,-26.900
euribor3m,41188.0,3.621291,1.734447,0.634,1.344,4.857,4.961,5.045
nr.employed,41188.0,5167.035911,72.251528,4963.600,5099.100,5191.000,5228.100,5228.100


>### Observaciones:
>
> - Tenemos 41188  Filas y 21 Columnas a analizar.
> - Los datos cuentan con 10 características númericas y 11 característica categóricas.
> - Observamos que tenemos a clientes con edad de 98 años. Habría que mirar si es un caso aislado.
> - Que tiene alguna llamada de un max de 82 min, aunque lo normal es que sea una media de 4 min aprox. El 75% de las llamadas es entorno a 5 min.
> - pdays es Número de días que transcurrieron desde la última campaña hasta que fue contactado que todos los valores del 25,5075... den 999 es extraño




## Limpieza de datos

In [6]:
df_inicio.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
41183    False
41184    False
41185    False
41186    False
41187    False
Length: 41188, dtype: bool

In [7]:
#Vemos en primer lugar que columnas tenemos con que nº de datos categoricos. 

df_inicio['job'].value_counts()

job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: count, dtype: int64

age. Edad del cliente (numérico)

job. Tipo de trabajo (categórico)

marital. Estado civil (categórico)

education. Nivel de educación (categórico)

default. ¿Tiene crédito actualmente? (categórico)

housing. ¿Tiene un préstamo de vivienda? (categórico)

loan. ¿Tiene un préstamo personal? (categórico)

contact. Tipo de comunicación de contacto (categórico)

month. Último mes en el que se le ha contactado (categórico)

day_of_week. Último día en el que se le ha contactado (categórico)

duration. Duración del contacto previo en segundos (numérico)

campaign. Número de contactos realizados durante esta campaña al cliente (numérico)

pdays. Número de días que transcurrieron desde la última campaña hasta que fue contactado (numérico)

previous. Número de contactos realizados durante la campaña anterior al cliente (numérico)

poutcome. Resultado de la campaña de marketing anterior (categórico)

emp.var.rate. Tasa de variación del empleo. Indicador trimestral (numérico)

cons.price.idx. Índice de precios al consumidor. Indicador mensual (numérico)

cons.conf.idx. Índice de confianza del consumidor. Indicador mensual (numérico)

euribor3m. Tasa EURIBOR 3 meses. Indicador diario (numérico)

nr.employed. Número de empleados. Indicador trimestral (numérico)

y. TARGET. El cliente contrata un depósito a largo plazo o no (categórico)

In [8]:
valor_cero = (df_inicio['previous'] == 0).sum()

valor_cero

np.int64(35563)